In [43]:
import gizmo_analysis as gizmo
import h5py
import halo_analysis as halo
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate as interp
import scipy.signal as signal
import utilities as ut
from gc_utils import get_descendants_halt, iteration_name, main_prog_halt, snapshot_name  # type: ignore


In [44]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"
fire_dir = sim_dir + sim + "/" + sim + "_res7100"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

snap_dir = fire_dir + "/snapshot_times.txt"
snap_data = pd.read_table(snap_dir, comment="#", header=None, sep=r"\s+")
snap_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

snap_pub_dir = sim_dir + "/snapshot_times_public.txt"
snap_pub_data = pd.read_table(snap_pub_dir, comment="#", header=None, sep=r"\s+")
snap_pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

In [45]:
snap = 77

hal = halo.io.IO.read_catalogs("index", snap, fire_dir, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478

* read 9832 halos from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/halo_077.hdf5

# in halo_analysis.halo_io.Particle():
* read 9832 halos, 301 have star particles, from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/star_077.hdf5
* assigning primary host and coordinates wrt it to halo catalog...  finished


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/track/host_coordinates.hdf5
  host1 position = (43057.26, 42130.64, 44311.36) [kpc comoving]
  host1 velocity = (-11.0, 39.1, 70.4) [km/s]
  host1 axis ratios = (0.53, 0.65, 0.81)


In [46]:
part = gizmo.io.Read.read_snapshots(
    ["dark", "star"], "index", snap, simulation_directory=fire_dir, assign_hosts_rotation=True
)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478




# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_077/snapshot_077.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70388512 particles
    star      (id = 4): 125821 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star']

* reading particles from:
    snapshot_077.0.hdf5
    snapshot_077.1.hdf5
    snapshot_077.2.hdf5
    snapshot_077.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/track/host_coordinates.hdf5
  host1 position = (43057.26, 42130

# LETS DO A TEST TO JUST CONFIRM THE PROCESS

In [47]:
host_idx = np.where(hal.prop("host.distance.principal.total") == 0)[0][0]
hal["position"][host_idx]

array([43056.184, 42130.63 , 44310.984], dtype=float32)

In [48]:
# second largest galaxy
galaxy_b_idx = np.where(hal["mass"] == np.flip(np.sort(hal["mass"]))[1])[0][0]

# dark matter close to the centre of galaxy_B
particle_idx = 31986394

## Start with position

In [49]:
# this is what I need to match
hal.prop("host.distance.principal", galaxy_b_idx)

array([ 100.06183, -256.75357, -249.67384], dtype=float32)

In [50]:
rotate_tensor = hal.host["rotation"][0]
unrotate_tensor = rotate_tensor.transpose()

In [51]:
ut.coordinate.get_coordinates_rotated(hal.prop("host.distance.principal", galaxy_b_idx), unrotate_tensor)

array([ 143.40086, -321.53067,  119.69134], dtype=float32)

In [52]:
# position of galaxy B in reference to galaxy A

galaxy_b_pos_a_unrotated = ut.coordinate.get_distances(
    hal["position"][galaxy_b_idx],
    hal["position"][host_idx],
    hal.info["box.length"],
    hal.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

galaxy_b_pos_a_unrotated

array([ 143.40086, -321.53067,  119.69134], dtype=float32)

In [53]:
galaxy_b_pos_a_rotated = ut.coordinate.get_coordinates_rotated(galaxy_b_pos_a_unrotated, rotate_tensor)
galaxy_b_pos_a_rotated

array([ 100.06183, -256.75357, -249.67384], dtype=float32)

In [54]:
# now lets do the same for the particle

# this is what I need to match
part["dark"].prop("host.distance.principal", particle_idx)

array([  99.87586756, -256.63107867, -249.68264204])

In [55]:
ut.coordinate.get_coordinates_rotated(
    part["dark"].prop("host.distance.principal", particle_idx), unrotate_tensor
)

array([ 143.22875382, -321.51075297,  119.5511743 ])

In [56]:
# position of galaxy B in reference to galaxy A

particle_pos_a_unrotated = ut.coordinate.get_distances(
    part["dark"]["position"][particle_idx],
    hal["position"][host_idx],
    hal.info["box.length"],
    hal.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

particle_pos_a_unrotated

array([ 143.42579157, -321.50985401,  119.62052775])

In [57]:
particle_pos_a_rotated = ut.coordinate.get_coordinates_rotated(particle_pos_a_unrotated, rotate_tensor)
particle_pos_a_rotated

array([ 100.07836198, -256.6806758 , -249.69575282])

In [58]:
# now lets get the distance between galaxy a and star particle

test_rotate = ut.coordinate.get_distances(
    particle_pos_a_rotated,
    galaxy_b_pos_a_rotated,
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

test_rotate, np.linalg.norm(test_rotate)

(array([ 0.003018  ,  0.01330619, -0.00399934]), 0.014218215084622933)

In [59]:
test_unrotate = ut.coordinate.get_distances(
    particle_pos_a_unrotated,
    galaxy_b_pos_a_unrotated,
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

test_unrotate, np.linalg.norm(test_unrotate)

(array([ 0.00455034,  0.00379978, -0.01292561]), 0.014220237125981812)

## And velocity

In [60]:
# this is what I need to match
hal.prop("host.velocity.principal", galaxy_b_idx)

array([ -17.265121, -116.853424, -138.86247 ], dtype=float32)

In [61]:
ut.coordinate.get_coordinates_rotated(hal.prop("host.velocity.principal", galaxy_b_idx), unrotate_tensor)

array([  10.334517, -179.11452 ,   32.35399 ], dtype=float32)

In [62]:
galaxy_b_vel_a_unrotated = ut.coordinate.get_velocity_differences(
    hal["velocity"][galaxy_b_idx],
    hal["velocity"][host_idx],
    hal["position"][galaxy_b_idx],
    hal["position"][host_idx],
    hal.info["box.length"],
    hal.snapshot["scalefactor"],
    hal.snapshot["time.hubble"],
    total_velocity=False,
)

galaxy_b_vel_a_unrotated

array([  10.334518, -179.11452 ,   32.353985], dtype=float32)

In [63]:
galaxy_b_vel_a_rotated = ut.coordinate.get_coordinates_rotated(galaxy_b_vel_a_unrotated, rotate_tensor)
galaxy_b_vel_a_rotated

array([ -17.265121, -116.853424, -138.86247 ], dtype=float32)

In [64]:
# now lets do the same for the particle

# this is what I need to match
part["dark"].prop("host.velocity.principal", particle_idx)

array([ -47.19445, -149.82497, -189.19464], dtype=float32)

In [65]:
ut.coordinate.get_coordinates_rotated(
    part["dark"].prop("host.velocity.principal", particle_idx), unrotate_tensor
)

array([  -8.407727, -243.65054 ,   32.143   ], dtype=float32)

In [66]:
particle_vel_a_unrotated = ut.coordinate.get_velocity_differences(
    part["dark"]["velocity"][particle_idx],
    hal.host["velocity"],
    part["dark"]["position"][particle_idx],
    hal.host["position"],
    hal.info["box.length"],
    hal.snapshot["scalefactor"],
    hal.snapshot["time.hubble"],
    total_velocity=False,
)

particle_vel_a_unrotated

array([[  -8.407779, -243.65042 ,   32.14295 ]], dtype=float32)

In [67]:
particle_vel_a_rotated = ut.coordinate.get_coordinates_rotated(particle_vel_a_unrotated, rotate_tensor)
particle_vel_a_rotated

array([[ -47.19449, -149.82487, -189.19455]], dtype=float32)

In [68]:
# now lets get the distance between galaxy a and star particle

test_rotate = ut.coordinate.get_velocity_differences(
    particle_vel_a_rotated,
    galaxy_b_vel_a_rotated,
    particle_pos_a_rotated,
    galaxy_b_pos_a_rotated,
    hal.info["box.length"],
    hal.snapshot["scalefactor"],
    hal.snapshot["time.hubble"],
    total_velocity=False,
)

test_rotate, np.linalg.norm(test_rotate)

(array([[-29.927938, -32.965153, -50.33397 ]], dtype=float32), 67.200386)

In [ ]:
(
    ut.coordinate.get_coordinates_rotated(test_rotate, unrotate_tensor),
    np.linalg.norm(ut.coordinate.get_coordinates_rotated(test_rotate, unrotate_tensor)),
)

array([[-18.74014   , -64.53411   ,  -0.21714869]], dtype=float32)

In [69]:
test_unrotate = ut.coordinate.get_velocity_differences(
    particle_vel_a_unrotated,
    galaxy_b_vel_a_unrotated,
    particle_pos_a_unrotated,
    galaxy_b_pos_a_unrotated,
    hal.info["box.length"],
    hal.snapshot["scalefactor"],
    hal.snapshot["time.hubble"],
    total_velocity=False,
)

test_unrotate, np.linalg.norm(test_unrotate)

(array([[-18.740145  , -64.5341    ,  -0.21715374]], dtype=float32), 67.20038)

DONT FORGET WILL NEED TO DO VELOCITY DISPERSION